
# Burger Joint in Madrid

## 1. Introduction / business problem

Burgers although reviled for being considered an icon of fast food, has earned a place in our gastronomy and they are more fashionable than ever. 

In Madrid, if someone is looking to open a burger joint, the question is, where would you recommend to open it? The background of the problem is that in order for a burger joint to be profitable, there must be enough customers, and in order to have enough customers, it is not worth setting up one in the immediate proximity of existing ones.

Let's also make sure that the location to open the local should be a place with the higher possible customers. The new owner should care about this problem because the location of the new burger joint has a significant impact on the expected returns.

## 2. Data Preparation

We will use a CSV file with data extracted from Wikipedia with the information of each of the 21 districts in Madrid: Population, Latitute and Longitude.
Source: https://en.wikipedia.org/wiki/Districts_of_Madrid

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
!pip install folium
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

     |████████████████████████████████| 102kB 6.8MB/s ta 0:00:011


We will use a CSV file with data from Wikipedia with the information of each of the 21 districts in Madrid: Population, Latitude and Longitude

In [2]:
# The code was removed by Watson Studio for sharing.

,Number,District,latitude,longitude,population
0,1,Centro,40.415347,-3.707371,131928
1,2,Arganzuela,40.402733,-3.695403,151965
2,3,Retiro,40.413170,-3.683070,118516
3,4,Salamanca,40.430000,-3.677778,143800
4,5,Chamartin,40.453333,-3.677500,143424
5,6,Tetuan,40.460556,-3.700000,153789
6,7,Chamberi,40.432792,-3.697186,137401
7,8,Fuencarral-El Pardo,40.478611,-3.709722,238756
8,9,Moncloa-Aravaca,40.435151,-3.718765,116903
9,10,Latina,40.402461,-3.741294,233808


Add the population weighttage for each district

In [3]:

total_population = df_madrid['population'].sum()
#print (total_population)

# the higher populated district will have the higher score
df_madrid['Population Score'] = df_madrid['population']/total_population*100

# sort the dataframe based on the descending population score
df_madrid = df_madrid.sort_values('Population Score', ascending=False)
df_madrid.head(10)

,Number,District,latitude,longitude,population,Population Score
10,11,Carabanchel,40.383669,-3.727989,243998,7.667649
7,8,Fuencarral-El Pardo,40.478611,-3.709722,238756,7.502919
9,10,Latina,40.402461,-3.741294,233808,7.347427
12,13,Puente de Vallecas,40.398204,-3.669059,227595,7.152184
14,15,Ciudad Lineal,40.450000,-3.650000,212529,6.678734
15,16,Hortaleza,40.469457,-3.640482,180462,5.671027
19,20,San Blas-Canillejas,40.426001,-3.612764,154357,4.850676
5,6,Tetuan,40.460556,-3.700000,153789,4.832827
1,2,Arganzuela,40.402733,-3.695403,151965,4.775507
3,4,Salamanca,40.430000,-3.677778,143800,4.518922


Location of the districts in the Madrid Map

In [6]:
address = "Madrid, ON"
madrid_data = df_madrid
geolocator = Nominatim(user_agent="madrid_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid city are 40.457473750000005, -3.4696396792893456.


In [7]:
# create map of Madrid using latitude and longitude values
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=10)
map_madrid

In [8]:
for lat, lng, District in zip(
        df_madrid['latitude'], 
        df_madrid['longitude'], 
        df_madrid['District']): 
    label = '{}'.format(District)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  

map_madrid

Use of Foursquare API to find out the venue category.

In [9]:
# The code was removed by Watson Studio for sharing.

In [10]:
df_madrid.loc[1, 'District']
df_madrid.head(1)

,Number,District,latitude,longitude,population,Population Score
10,11,Carabanchel,40.383669,-3.727989,243998,7.667649


In [11]:
district_latitude = df_madrid.loc[1, 'latitude'] # district latitude value
district_longitude = df_madrid.loc[1, 'longitude'] # district longitude value

district_name = df_madrid.loc[1, 'District'] # district name

print('Latitude and longitude values of {} are {}, {}.'.format(district_name, 
                                                               district_latitude, 
                                                               district_longitude))

Latitude and longitude values of Arganzuela are 40.402733000000005, -3.6954029999999998.


In [12]:
# explore the top 100 venues
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    district_latitude, 
    district_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=I3CGPNI11NL3KKOQFBWU1U2GM1LPD3LYTAHSHI2KNLH2MREH&client_secret=JRALNZW3PRMA1R4B5AS4V1TLGN0EGTA24ZOV2FLRSEBGIHLZ&v=20180604&ll=40.402733000000005,-3.6954029999999998&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f64d7a5fe90f72a3a12c259'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Madrid',
  'headerFullLocation': 'Madrid',
  'headerLocationGranularity': 'city',
  'totalResults': 82,
  'suggestedBounds': {'ne': {'lat': 40.40723300450001,
    'lng': -3.689504695496171},
   'sw': {'lat': 40.3982329955, 'lng': -3.7013013045038283}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d55a13048ea6ea8fb42d7a3',
       'name': 'Pui Thai Tapas',
       'location': {'address': 'C. José Antonio de Armona, 7',
        'lat': 40.40447537896806,
        'lng': -3.696691614807786,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.40447537896806,


In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pui Thai Tapas,Thai Restaurant,40.404475,-3.696692
1,Cervecissimus Delicias,Beer Bar,40.404211,-3.694658
2,Mercado de Santa María de la Cabeza,Farmers Market,40.402860,-3.696819
3,Havana Blues,Cuban Restaurant,40.402050,-3.698488
4,Basic-Fit Delicias,Gym,40.404225,-3.693834


In [16]:

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

82 venues were returned by Foursquare.


In [17]:
# Explore districts
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
madrid_venues = getNearbyVenues(names=df_madrid['District'],
                                   latitudes=df_madrid['latitude'],
                                   longitudes=df_madrid['longitude']
                                  )

print(madrid_venues.shape)
madrid_venues.head(10)

Carabanchel
Fuencarral-El Pardo
Latina
Puente de Vallecas
Ciudad Lineal
Hortaleza
San Blas-Canillejas
Tetuan
Arganzuela
Salamanca
Chamartin
Villaverde
Chamberi
Usera
Centro
Retiro
Moncloa-Aravaca
Villa de Vallecas
Moratalaz
Vicalvaro
Barajas
(711, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Carabanchel,40.383669,-3.727989,La Piazzola,40.385917,-3.724067,Burger Joint
1,Carabanchel,40.383669,-3.727989,Burger King,40.381050,-3.728027,Fast Food Restaurant
2,Carabanchel,40.383669,-3.727989,La Nueva Pocha,40.386409,-3.728038,Tapas Restaurant
3,Carabanchel,40.383669,-3.727989,Telepizza,40.381239,-3.728458,Pizza Place
4,Carabanchel,40.383669,-3.727989,La Ventura,40.386655,-3.728057,Nightclub
5,Carabanchel,40.383669,-3.727989,Metro Abrantes,40.380950,-3.727927,Metro Station
6,Carabanchel,40.383669,-3.727989,Glorieta del Valle de Oro,40.387673,-3.730461,Plaza
7,Carabanchel,40.383669,-3.727989,Migas,40.380106,-3.727942,Bakery
8,Carabanchel,40.383669,-3.727989,Campos de Futbol Ernesto Cotorruelo,40.380795,-3.724066,Soccer Field
9,Fuencarral-El Pardo,40.478611,-3.709722,Disney Store,40.479557,-3.706362,Toy / Game Store


In [19]:
# find out how many unique categories
madrid_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Arganzuela,82,82,82,82,82,82
Barajas,29,29,29,29,29,29
Carabanchel,9,9,9,9,9,9
Centro,96,96,96,96,96,96
Chamartin,53,53,53,53,53,53
Chamberi,100,100,100,100,100,100
Ciudad Lineal,27,27,27,27,27,27
Fuencarral-El Pardo,39,39,39,39,39,39
Hortaleza,19,19,19,19,19,19


In [20]:
# analyze each district

# one hot encoding
madrid_onehot = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add district column back to dataframe
madrid_onehot['District'] = madrid_venues['District'] 

# move district column to the first column
fixed_columns = [madrid_onehot.columns[-1]] + list(madrid_onehot.columns[:-1])
madrid_onehot = madrid_onehot[fixed_columns]

madrid_onehot.head()

,District,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trade School,Train Station,Udon Restaurant,Video Game Store,Wine Bar,Wine Shop,Women's Store
0,Carabanchel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Carabanchel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Carabanchel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Carabanchel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Carabanchel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# group rows by district and by taking the mean of the frequency of occurrence of each category
madrid_grouped = madrid_onehot.groupby('District').mean().reset_index()
madrid_grouped.head(10)

,District,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trade School,Train Station,Udon Restaurant,Video Game Store,Wine Bar,Wine Shop,Women's Store
0,Arganzuela,0.000000,0.02439,0.000000,0.00,0.000000,0.0,0.012195,0.0,0.00,...,0.012195,0.012195,0.000000,0.012195,0.012195,0.000000,0.000000,0.000000,0.0,0.0
1,Barajas,0.000000,0.00000,0.034483,0.00,0.000000,0.0,0.034483,0.0,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.0,0.0
2,Carabanchel,0.000000,0.00000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,Centro,0.010417,0.00000,0.000000,0.00,0.010417,0.0,0.000000,0.0,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010417,0.0,0.0
4,Chamartin,0.000000,0.00000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
5,Chamberi,0.010000,0.00000,0.000000,0.01,0.010000,0.0,0.020000,0.0,0.01,...,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0,0.0
6,Ciudad Lineal,0.000000,0.00000,0.074074,0.00,0.000000,0.0,0.000000,0.0,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
7,Fuencarral-El Pardo,0.025641,0.00000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,...,0.000000,0.000000,0.025641,0.000000,0.000000,0.025641,0.025641,0.000000,0.0,0.0
8,Hortaleza,0.000000,0.00000,0.052632,0.00,0.000000,0.0,0.000000,0.0,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
9,Latina,0.000000,0.00000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,...,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.0,0.0


Find out the top 10 most common venues in the neighbourhood.

In [22]:
# print each district along with the top 5 most common venues
num_top_venues = 5

for hood in madrid_grouped['District']:
    #print("----"+hood+"----")
    temp = madrid_grouped[madrid_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [23]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
madrid_grouped.head()

,District,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trade School,Train Station,Udon Restaurant,Video Game Store,Wine Bar,Wine Shop,Women's Store
0,Arganzuela,0.000000,0.02439,0.000000,0.0,0.000000,0.0,0.012195,0.0,0.0,...,0.012195,0.012195,0.0,0.012195,0.012195,0.0,0.0,0.000000,0.0,0.0
1,Barajas,0.000000,0.00000,0.034483,0.0,0.000000,0.0,0.034483,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.034483,0.0,0.0
2,Carabanchel,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
3,Centro,0.010417,0.00000,0.000000,0.0,0.010417,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.010417,0.0,0.0
4,Chamartin,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0


 Explore the top 5 venues for each district.

In [25]:
# create the new dataframe and display the top 5 venues for each district.
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
District_venues_sorted = pd.DataFrame(columns=columns)
District_venues_sorted['District'] = madrid_grouped['District']

for ind in np.arange(madrid_grouped.shape[0]):
    District_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

District_venues_sorted.head(21)

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Arganzuela,Restaurant,Spanish Restaurant,Grocery Store,Bakery,Tapas Restaurant
1,Barajas,Hotel,Restaurant,Spanish Restaurant,Coffee Shop,Tapas Restaurant
2,Carabanchel,Soccer Field,Plaza,Burger Joint,Fast Food Restaurant,Tapas Restaurant
3,Centro,Plaza,Tapas Restaurant,Spanish Restaurant,Hostel,Bistro
4,Chamartin,Restaurant,Spanish Restaurant,Grocery Store,Bakery,Tapas Restaurant
5,Chamberi,Spanish Restaurant,Bar,Restaurant,Brewery,Japanese Restaurant
6,Ciudad Lineal,Spanish Restaurant,Gastropub,Burger Joint,Argentinian Restaurant,Restaurant
7,Fuencarral-El Pardo,Clothing Store,Restaurant,Burger Joint,Italian Restaurant,Fast Food Restaurant
8,Hortaleza,Breakfast Spot,Supermarket,Pizza Place,Spanish Restaurant,Donut Shop
9,Latina,Pizza Place,Park,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant


Add the burger joint weighted score for each district. Also add score for fast food restaurants as they are similar type of food.

In [26]:
District_venues_sorted['Burger Score'] = 0
venue1 = 'Burger'
venue_count = 0

# award the score based on the common findings
District_venues_sorted.loc[District_venues_sorted['1st Most Common Venue'].str.contains(venue1), ['Burger Score']] = 1.0
District_venues_sorted.loc[District_venues_sorted['2nd Most Common Venue'].str.contains(venue1), ['Burger Score']] = 0.8
District_venues_sorted.loc[District_venues_sorted['3rd Most Common Venue'].str.contains(venue1), ['Burger Score']] = 0.6
District_venues_sorted.loc[District_venues_sorted['4th Most Common Venue'].str.contains(venue1), ['Burger Score']] = 0.4
District_venues_sorted.loc[District_venues_sorted['5th Most Common Venue'].str.contains(venue1), ['Burger Score']] = 0.2



In [27]:
District_venues_sorted['Fast Food Score'] = 0
venue2 = 'Fast Food'
venue_count = 0

# award the score based on the common findings
District_venues_sorted.loc[District_venues_sorted['1st Most Common Venue'].str.contains(venue2), ['Fast Food Score']] = 1.0
District_venues_sorted.loc[District_venues_sorted['2nd Most Common Venue'].str.contains(venue2), ['Fast Food Score']] = 0.8
District_venues_sorted.loc[District_venues_sorted['3rd Most Common Venue'].str.contains(venue2), ['Fast Food Score']] = 0.6
District_venues_sorted.loc[District_venues_sorted['4th Most Common Venue'].str.contains(venue2), ['Fast Food Score']] = 0.4
District_venues_sorted.loc[District_venues_sorted['5th Most Common Venue'].str.contains(venue2), ['Fast Food Score']] = 0.2

# merge the two dataframes
merged_data = pd.merge(df_madrid, District_venues_sorted, on='District')

# clean up the merged dataset
merged_data = merged_data.drop('1st Most Common Venue', 1)
merged_data = merged_data.drop('2nd Most Common Venue', 1)
merged_data = merged_data.drop('3rd Most Common Venue', 1)
merged_data = merged_data.drop('4th Most Common Venue', 1)
merged_data = merged_data.drop('5th Most Common Venue', 1)

merged_data.head(10)



,Number,District,latitude,longitude,population,Population Score,Burger Score,Fast Food Score
0,11,Carabanchel,40.383669,-3.727989,243998,7.667649,0.6,0.4
1,8,Fuencarral-El Pardo,40.478611,-3.709722,238756,7.502919,0.6,0.2
2,10,Latina,40.402461,-3.741294,233808,7.347427,0.0,0.4
3,13,Puente de Vallecas,40.398204,-3.669059,227595,7.152184,0.0,1.0
4,15,Ciudad Lineal,40.450000,-3.650000,212529,6.678734,0.6,0.0
5,16,Hortaleza,40.469457,-3.640482,180462,5.671027,0.0,0.0
6,20,San Blas-Canillejas,40.426001,-3.612764,154357,4.850676,0.0,0.0
7,6,Tetuan,40.460556,-3.700000,153789,4.832827,0.0,0.0
8,2,Arganzuela,40.402733,-3.695403,151965,4.775507,0.0,0.0
9,4,Salamanca,40.430000,-3.677778,143800,4.518922,0.2,0.0


Calculate the total weighted score for each district.

In [28]:

# considering this to be a fair weightage on the three parameters
population_weight = 0.5
burger_weight = 0.3
fast_food_weight = 0.2

# calculate the total score
merged_data['Total Score'] = ((merged_data['Population Score']*population_weight) + (merged_data['Burger Score']*burger_weight) + (merged_data['Fast Food Score']*fast_food_weight))

# sort the dataframe based on the descending total score
merged_data = merged_data.sort_values('Total Score', ascending=False)

## 3. Metodology: Cluster the districs using K-Clustering.

In [29]:
# set number of clusters
kclusters = 4

#madrid_grouped_clustering = madrid_grouped.drop('District', 1)

# clean up data further
madrid_grouped_clustering = merged_data.drop('District', 1)
madrid_grouped_clustering = madrid_grouped_clustering.drop('Number', 1)

madrid_grouped_clustering.head(10)

,latitude,longitude,population,Population Score,Burger Score,Fast Food Score,Total Score
0,40.383669,-3.727989,243998,7.667649,0.6,0.4,4.093824
1,40.478611,-3.709722,238756,7.502919,0.6,0.2,3.971459
3,40.398204,-3.669059,227595,7.152184,0.0,1.0,3.776092
2,40.402461,-3.741294,233808,7.347427,0.0,0.4,3.753714
4,40.450000,-3.650000,212529,6.678734,0.6,0.0,3.519367
5,40.469457,-3.640482,180462,5.671027,0.0,0.0,2.835513
6,40.426001,-3.612764,154357,4.850676,0.0,0.0,2.425338
7,40.460556,-3.700000,153789,4.832827,0.0,0.0,2.416413
8,40.402733,-3.695403,151965,4.775507,0.0,0.0,2.387754
9,40.430000,-3.677778,143800,4.518922,0.2,0.0,2.319461


In [30]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 2, 2, 2, 2, 2], dtype=int32)

In [31]:
# empty the invalid data
#merged_data['District'] = District_venues_sorted['District']
merged_data.isnull().sum()
merged_data.dropna(how = 'any', axis = 0, inplace = True)
merged_data = merged_data.drop('Number', 1)
print(merged_data.shape)
print(District_venues_sorted.shape)

(21, 8)
(21, 8)


In [32]:
merged_data.head()

,District,latitude,longitude,population,Population Score,Burger Score,Fast Food Score,Total Score
0,Carabanchel,40.383669,-3.727989,243998,7.667649,0.6,0.4,4.093824
1,Fuencarral-El Pardo,40.478611,-3.709722,238756,7.502919,0.6,0.2,3.971459
3,Puente de Vallecas,40.398204,-3.669059,227595,7.152184,0.0,1.0,3.776092
2,Latina,40.402461,-3.741294,233808,7.347427,0.0,0.4,3.753714
4,Ciudad Lineal,40.450000,-3.650000,212529,6.678734,0.6,0.0,3.519367


In [33]:
District_venues_sorted = District_venues_sorted.drop('Burger Score', 1)
District_venues_sorted = District_venues_sorted.drop('Fast Food Score', 1)
District_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Arganzuela,Restaurant,Spanish Restaurant,Grocery Store,Bakery,Tapas Restaurant
1,Barajas,Hotel,Restaurant,Spanish Restaurant,Coffee Shop,Tapas Restaurant
2,Carabanchel,Soccer Field,Plaza,Burger Joint,Fast Food Restaurant,Tapas Restaurant
3,Centro,Plaza,Tapas Restaurant,Spanish Restaurant,Hostel,Bistro
4,Chamartin,Restaurant,Spanish Restaurant,Grocery Store,Bakery,Tapas Restaurant


In [34]:
madrid_merged = df_madrid
# add clustering labels
merged_data['Cluster Labels'] = kmeans.labels_

# merge madrid_grouped with toronto_data to add latitude/longitude for each district
madrid_merged = merged_data.join(District_venues_sorted.set_index('District'), on='District')

madrid_merged.head(10) 

,District,latitude,longitude,population,Population Score,Burger Score,Fast Food Score,Total Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Carabanchel,40.383669,-3.727989,243998,7.667649,0.6,0.4,4.093824,0,Soccer Field,Plaza,Burger Joint,Fast Food Restaurant,Tapas Restaurant
1,Fuencarral-El Pardo,40.478611,-3.709722,238756,7.502919,0.6,0.2,3.971459,0,Clothing Store,Restaurant,Burger Joint,Italian Restaurant,Fast Food Restaurant
3,Puente de Vallecas,40.398204,-3.669059,227595,7.152184,0.0,1.0,3.776092,0,Fast Food Restaurant,Supermarket,Grocery Store,Hotel,Café
2,Latina,40.402461,-3.741294,233808,7.347427,0.0,0.4,3.753714,0,Pizza Place,Park,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant
4,Ciudad Lineal,40.450000,-3.650000,212529,6.678734,0.6,0.0,3.519367,0,Spanish Restaurant,Gastropub,Burger Joint,Argentinian Restaurant,Restaurant
5,Hortaleza,40.469457,-3.640482,180462,5.671027,0.0,0.0,2.835513,2,Breakfast Spot,Supermarket,Pizza Place,Spanish Restaurant,Donut Shop
6,San Blas-Canillejas,40.426001,-3.612764,154357,4.850676,0.0,0.0,2.425338,2,Metro Station,Supermarket,Snack Place,Gym,Grocery Store
7,Tetuan,40.460556,-3.700000,153789,4.832827,0.0,0.0,2.416413,2,Spanish Restaurant,Supermarket,Bakery,Grocery Store,Brazilian Restaurant
8,Arganzuela,40.402733,-3.695403,151965,4.775507,0.0,0.0,2.387754,2,Restaurant,Spanish Restaurant,Grocery Store,Bakery,Tapas Restaurant
9,Salamanca,40.430000,-3.677778,143800,4.518922,0.2,0.0,2.319461,2,Spanish Restaurant,Restaurant,Coffee Shop,Bakery,Burger Joint


Draw the cluster map.

In [35]:
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(madrid_merged['latitude'], madrid_merged['longitude'], madrid_merged['District'], madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Results

In [36]:
# display the top 5 districts
target = merged_data.head(5)
target.head()

,District,latitude,longitude,population,Population Score,Burger Score,Fast Food Score,Total Score,Cluster Labels
0,Carabanchel,40.383669,-3.727989,243998,7.667649,0.6,0.4,4.093824,0
1,Fuencarral-El Pardo,40.478611,-3.709722,238756,7.502919,0.6,0.2,3.971459,0
3,Puente de Vallecas,40.398204,-3.669059,227595,7.152184,0.0,1.0,3.776092,0
2,Latina,40.402461,-3.741294,233808,7.347427,0.0,0.4,3.753714,0
4,Ciudad Lineal,40.450000,-3.650000,212529,6.678734,0.6,0.0,3.519367,0


In [37]:
# Cluster 1 
madrid_merged.loc[madrid_merged['Cluster Labels'] == 0, madrid_merged.columns[[0] + [1] + list(range(4, madrid_merged.shape[1]))]].sort_values('Population Score', ascending=False).head(5)

,District,latitude,Population Score,Burger Score,Fast Food Score,Total Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Carabanchel,40.383669,7.667649,0.6,0.4,4.093824,0,Soccer Field,Plaza,Burger Joint,Fast Food Restaurant,Tapas Restaurant
1,Fuencarral-El Pardo,40.478611,7.502919,0.6,0.2,3.971459,0,Clothing Store,Restaurant,Burger Joint,Italian Restaurant,Fast Food Restaurant
2,Latina,40.402461,7.347427,0.0,0.4,3.753714,0,Pizza Place,Park,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant
3,Puente de Vallecas,40.398204,7.152184,0.0,1.0,3.776092,0,Fast Food Restaurant,Supermarket,Grocery Store,Hotel,Café
4,Ciudad Lineal,40.450000,6.678734,0.6,0.0,3.519367,0,Spanish Restaurant,Gastropub,Burger Joint,Argentinian Restaurant,Restaurant


In [38]:
# Cluster 2 
madrid_merged.loc[madrid_merged['Cluster Labels'] == 1, madrid_merged.columns[[0] + [1] + list(range(4, madrid_merged.shape[1]))]].sort_values('Population Score', ascending=False).head(5)

,District,latitude,Population Score,Burger Score,Fast Food Score,Total Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
15,Retiro,40.413170,3.724371,0.0,0.0,1.862185,1,Garden,Café,Plaza,Park,Monument / Landmark
16,Moncloa-Aravaca,40.435151,3.673682,0.0,0.0,1.836841,1,Tapas Restaurant,Pub,Spanish Restaurant,Restaurant,Ice Cream Shop
17,Villa de Vallecas,40.379600,3.281435,0.0,0.0,1.640717,1,Soccer Field,Park,Food,Train Station,Spanish Restaurant
18,Moratalaz,40.409869,2.960145,0.0,0.0,1.480073,1,Soccer Field,Nightclub,Brewery,Café,Falafel Restaurant


In [39]:
# Cluster 3 
madrid_merged.loc[madrid_merged['Cluster Labels'] == 2, madrid_merged.columns[[0] + [1] + list(range(4, madrid_merged.shape[1]))]].sort_values('Population Score', ascending=False).head(5)

,District,latitude,Population Score,Burger Score,Fast Food Score,Total Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Hortaleza,40.469457,5.671027,0.0,0.0,2.835513,2,Breakfast Spot,Supermarket,Pizza Place,Spanish Restaurant,Donut Shop
6,San Blas-Canillejas,40.426001,4.850676,0.0,0.0,2.425338,2,Metro Station,Supermarket,Snack Place,Gym,Grocery Store
7,Tetuan,40.460556,4.832827,0.0,0.0,2.416413,2,Spanish Restaurant,Supermarket,Bakery,Grocery Store,Brazilian Restaurant
8,Arganzuela,40.402733,4.775507,0.0,0.0,2.387754,2,Restaurant,Spanish Restaurant,Grocery Store,Bakery,Tapas Restaurant
9,Salamanca,40.430000,4.518922,0.2,0.0,2.319461,2,Spanish Restaurant,Restaurant,Coffee Shop,Bakery,Burger Joint


In [40]:
# Cluster 4 
madrid_merged.loc[madrid_merged['Cluster Labels'] == 3, madrid_merged.columns[[0] + [1] + list(range(4, madrid_merged.shape[1]))]].sort_values('Population Score', ascending=False).head(5)

,District,latitude,Population Score,Burger Score,Fast Food Score,Total Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,Vicalvaro,40.404200,2.201356,0.0,0.0,1.100678,3,Pizza Place,Spanish Restaurant,Breakfast Spot,Café,Camera Store
20,Barajas,40.470196,1.473081,0.0,0.0,0.736540,3,Hotel,Restaurant,Spanish Restaurant,Coffee Shop,Tapas Restaurant


Selectin the best result from cluster 3: Hortaleza district, now we use a heatmap based kernel density estimation as a Folium plugin. With that we can have a possible good location for our new burger joint.

In [41]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180604',
    "section": "Burger Joint",
    "near": "Hortaleza",
    "radius": 1000,
    "limit": 50}
data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [42]:

d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [43]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('unknown', 'Current map view', 'Current map view')

In [44]:

d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 40.48959604202841, 'lng': -3.6115114660573138},
  'sw': {'lat': 40.45300180253878, 'lng': -3.6703163340006917}},
 69)

In [45]:
d["geocode"]

{'what': '',
 'where': 'hortaleza',
 'center': {'lat': 40.47444, 'lng': -3.6411},
 'displayString': 'Hortaleza, Madrid, Spain',
 'cc': 'ES',
 'slug': 'hortaleza-spain',
 'longId': '72057594041048571'}

In [46]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [47]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [48]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b5d76d3f964a520de5d29e3',
  'name': "A'Fenestella",
  'location': {'address': 'C. Torquemada, 25',
   'lat': 40.46818813882213,
   'lng': -3.6386468870295485,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.46818813882213,
     'lng': -3.6386468870295485}],
   'postalCode': '28043',
   'cc': 'ES',
   'neighborhood': 'Canillas',
   'city': 'Madrid',
   'state': 'Madrid',
   'country': 'España',
   'formattedAddress': ['C. Torquemada, 25', '28043 Madrid Madrid', 'España']},
  'categories': [{'id': '4bf58dd8d48988d1ca941735',
    'name': 'Pizza Place',
    'pluralName': 'Pizza Places',
    'shortName': 'Pizza',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4b5d76d3f964a520de5d29e3-0

In [49]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ecc1f349a52a680864d0b10',
  'name': 'Zurita',
  'location': {'address': 'Calle de Santa Susana, 17, 28033 Madrid',
   'lat': 40.47673951128428,
   'lng': -3.646470237514776,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.47673951128428,
     'lng': -3.646470237514776}],
   'postalCode': '28033',
   'cc': 'ES',
   'city': 'Madrid',
   'state': 'Madrid',
   'country': 'España',
   'formattedAddress': ['Calle de Santa Susana, 17, 28033 Madrid',
    '28033 Madrid Madrid',
    'España']},
  'categories': [{'id': '4bf58dd8d48988d1db931735',
    'name': 'Tapas Restaurant',
    'pluralName': 'Tapas Restaurants',
    'shortName': 'Tapas',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tapas_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4ec

In [50]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["formattedAddress"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i burger joints" % len(df))
df.head()

found 32 burger joints


,uid,name,shortname,address,postalcode,lat,lng
0,4b5d76d3f964a520de5d29e3,A'Fenestella,Pizza,"[C. Torquemada, 25, 28043 Madrid Madrid, España]",28043,40.468188,-3.638647
1,4ecc1f349a52a680864d0b10,Zurita,Tapas,"[Calle de Santa Susana, 17, 28033 Madrid, 2803...",28033,40.476740,-3.646470
2,4b60f18bf964a520ab022ae3,Restaurante El Güero,Venezuelan,"[C. Emigrantes, 2, 28043 Madrid Madrid, España]",28043,40.465886,-3.635932
3,56c23c43cd104c422c6252b4,Everest Tandoori,Indian,"[Calle Pegaso 13, 28043 Madrid Madrid, España]",28043,40.464165,-3.649329
4,4c28edabfe6e2d7fba08543c,El Rincón De Malena,Spanish,"[Carretera de Canillas 144, 28043 Madrid Madri...",28043,40.465001,-3.638552


In [51]:
hortaleza = d["geocode"]["center"]
hortaleza

{'lat': 40.47444, 'lng': -3.6411}

In [52]:
from folium import plugins

# create map of Hortaleza using latitude and longitude values
map_hortaleza = folium.Map(location=[hortaleza["lat"], hortaleza["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_hortaleza)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_hortaleza.add_child(plugins.HeatMap(hm_data))

map_hortaleza

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [53]:
lat = 40.47444
lng = -3.6411
map_hortaleza = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Burger Joint!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_hortaleza)
map_hortaleza

## 4. Results

Based on the result above, the third cluster looks to offer a higher number of districts with good population score and low competition from others burger joints and fast food restaurants.

In this 3th cluster we recommend the Hortaleza district to open the first burger joint because it has the hihger population of all the cluster. Then to expand the bussiness the owner can consider the other districts from this cluster.

Then using folium we can see a possibly good location for our burger joint in the district. Avoiding been near the competition. 